<a href="https://colab.research.google.com/github/anindabitm/PDFM-use-case/blob/main/PDFM_embeddings_to_predict_PM2_5_in_US.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Predicting US PM2.5 levels using Google's Population Dynamics Foundation Model**

Useful Resources:
1. https://github.com/opengeos/GeoAI-Tutorials/blob/main/docs/PDFM/zillow_home_value.ipynb
2. https://github.com/google-research/population-dynamics/tree/master/notebooks

Acknowledgements:
This notebook is based on tutorials - [PDFM notebook](https://github.com/google-research/population-dynamics/tree/master/notebooks) and awesome tutorial by giswqs opengeos PDFM [zillow home price](https://github.com/opengeos/GeoAI-Tutorials/blob/main/docs/PDFM/zillow_home_value.ipynb)

In [ ]:
%%capture
!pip install leafmap

In [ ]:
# import libraries
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file

# Get US PM2.5 data
Link to data: https://usc-geohealth-hub-uscssi.hub.arcgis.com/documents/7fc448343d6643f3bb13157fd65aed4f/about

In [ ]:
df0 = pd.read_excel("/content/pm25_and_disparity.xlsx", sheet_name="data_part1")
df1 = pd.read_excel("/content/pm25_and_disparity.xlsx", sheet_name="data_part2")
df2 = pd.read_excel("/content/pm25_and_disparity.xlsx", sheet_name="data_part3")
df3 = pd.read_excel("/content/pm25_and_disparity.xlsx", sheet_name="data_part4")
df4 = pd.read_excel("/content/pm25_and_disparity.xlsx", sheet_name="data_part5")

# Process PM2.5 data

In [ ]:
df = pd.concat([df0, df1, df2, df3, df4], ignore_index=True)
df.head()

In [ ]:
df.shape

In [ ]:
df["zcta"].nunique()

In [ ]:
pm25_df = df.groupby(["zcta"]).mean()["pm25"]
pm25_df.head()

In [ ]:
pm25_df.dropna(axis=0, inplace=True)
pm25_df.head()

In [ ]:
pm25_df.index = pm25_df.index.astype(int)
print(pm25_df.shape)
pm25_df.head()

In [ ]:
pm25_df = pm25_df.reset_index(drop=False)  # Remove inplace=True
pm25_df.index = pm25_df["zcta"].apply(lambda x: f"zip/{x}")  # Access 'zcta' column
pm25_df.head()

# Request access to PDFM Embeddings

In [ ]:
!unzip /content/pdfm_embeddings.zip

In [ ]:
embeddings_file_path = "/content/pdfm_embeddings/v0/us/zcta_embeddings.csv"

In [ ]:
if not os.path.exists(embeddings_file_path):
    raise FileNotFoundError("Please request the embeddings from Google")

In [ ]:
zipcode_embeddings = pd.read_csv(embeddings_file_path).set_index("place")
zipcode_embeddings.head()

# Join PDFM embeddings and Groud Truth (PM2.5 data)

In [ ]:
data = pm25_df.join(zipcode_embeddings, how="inner")
data.head()

In [ ]:
data.shape

In [ ]:
embedding_features = [f"feature{x}" for x in range(330)]
label = "pm25"

In [ ]:
data = data.dropna(subset=[label])

# Split Train and Test Data

In [ ]:
data = data[embedding_features + [label]]
X = data[embedding_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit K-Nearest Neighbors Model

In [ ]:
k = 5
model = KNeighborsRegressor(n_neighbors=k)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
# Evaluate the model
metrics = evaluate_model(evaluation_df)
print(metrics)

# Evaluate K-Nearest Neighbors Model

In [ ]:
xy_lim = (0, 30)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted PM2.5",
    x_label="Actual PM2.5",
    y_label="Predicted PM2.5",
)

# Fit Random Forest Regressor model

In [ ]:
model = RandomForestRegressor(n_estimators=10, verbose=10, n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
# Evaluate the model
metrics = evaluate_model(evaluation_df)
print(metrics)

# Evaluate Random Forest Model

In [ ]:
xy_lim = (0, 30)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted PM2.5",
    x_label="Actual PM2.5",
    y_label="Predicted PM2.5",
)